In [1]:
%%capture
import os, importlib.util
!pip install --upgrade -qqq uv
if importlib.util.find_spec("torch") is None or "COLAB_" in "".join(os.environ.keys()):    
    try: import numpy, PIL; get_numpy = f"numpy=={numpy.__version__}"; get_pil = f"pillow=={PIL.__version__}"
    except: get_numpy = "numpy"; get_pil = "pillow"
    !uv pip install -qqq \
        "torch>=2.8.0" "triton>=3.4.0" {get_numpy} {get_pil} torchvision bitsandbytes "transformers==4.56.2" \
        "unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
        "unsloth[base] @ git+https://github.com/unslothai/unsloth" \
        git+https://github.com/triton-lang/triton.git@0add68262ab0a2e33b84524346cb27cbb2787356#subdirectory=python/triton_kernels
elif importlib.util.find_spec("unsloth") is None:
    !uv pip install -qqq unsloth
!uv pip install --upgrade --no-deps transformers==4.56.2 tokenizers trl==0.22.2 unsloth unsloth_zoo



def get_pooled_features(model, input_ids, attention_mask):
    """Extract features using the actual model structure"""
    with torch.no_grad():
        # Access the base model
        if hasattr(model, 'model'):
            base = model.model
        else:
            base = model
        
        # Try different ways to get embeddings
        embedding_layer = None
        
        # Method 1: Check for transformer.wte (GPT-2 style)
        if hasattr(base, 'transformer') and hasattr(base.transformer, 'wte'):
            embedding_layer = base.transformer.wte
            # print("Using: model.transformer.wte")
        # Method 2: Check for model.embed_tokens (LLaMA style)
        elif hasattr(base, 'model') and hasattr(base.model, 'embed_tokens'):
            embedding_layer = base.model.embed_tokens
            # print("Using: model.model.embed_tokens")
        # Method 3: Direct embed_tokens
        elif hasattr(base, 'embed_tokens'):
            embedding_layer = base.embed_tokens
            # print("Using: model.embed_tokens")
        # Method 4: Check decoder
        elif hasattr(base, 'model') and hasattr(base.model, 'decoder') and hasattr(base.model.decoder, 'embed_tokens'):
            embedding_layer = base.model.decoder.embed_tokens
            # print("Using: model.model.decoder.embed_tokens")
        else:
            raise ValueError("Cannot find embedding layer in model")
        
        # Get embeddings
        embeddings = embedding_layer(input_ids)
        
        # Mean pooling with attention mask
        mask_expanded = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
        sum_embeddings = torch.sum(embeddings * mask_expanded, 1)
        sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
        pooled = sum_embeddings / sum_mask
        
    return pooled

In [2]:
# ===== Load Base + DPO Model =====
from unsloth import FastLanguageModel
from peft import PeftModel

BASE_REPO = "Aharneish/finetuned_model-1"
DPO_REPO  = "Aharneish/dpo_out"

base, tokenizer = FastLanguageModel.from_pretrained(
    model_name=BASE_REPO,
    max_seq_length=1024,
    load_in_4bit=True,
)

dpo_model = PeftModel.from_pretrained(
    base,
    DPO_REPO,
    is_trainable=False
)

dpo_model.eval()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2026-01-22 05:08:25.121752: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769058505.486731      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769058505.594878      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769058506.512764      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769058506.512821      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769058506.512824      55 computation_placer.cc:177] computation placer alr

🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.3: Fast Gpt_Oss patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gpt_oss won't work! Using float32.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

...

adapter_config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:285: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/16.0M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/peft/peft_model.py:598: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layer

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PeftModelForCausalLM(
      (base_model): LoraModel(
        (model): GptOssForCausalLM(
          (model): GptOssModel(
            (embed_tokens): Embedding(201088, 2880, padding_idx=199999)
            (layers): ModuleList(
              (0-23): 24 x GptOssDecoderLayer(
                (self_attn): GptOssAttention(
                  (q_proj): lora.Linear4bit(
                    (base_layer): Linear4bit(in_features=2880, out_features=4096, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Identity()
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=2880, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=4096, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
          

In [3]:
# ===== EQ-Bench =====
from datasets import load_dataset
eqbench = load_dataset("pbevan11/EQ-Bench", split="validation")


README.md: 0.00B [00:00, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

Generating validation split:   0%|          | 0/171 [00:00<?, ? examples/s]

In [4]:
# ===== Ablations =====

# No DPO
model_no_dpo = base

# No emotion head
emotion_head = None

# No strategy head
strategy_head = None


In [5]:
# ===== Results Summary =====
import pandas as pd


In [6]:
import torch
from huggingface_hub import hf_hub_download
import torch.nn as nn

In [7]:
# 2. EmotionHead
class EmotionHead(nn.Module):
    def __init__(self, hidden_dim, num_labels=28):
        super().__init__()
        self.net = nn.Sequential(
            nn.LayerNorm(hidden_dim),
            nn.Dropout(0.1),
            nn.Linear(hidden_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, num_labels)
        )

    def forward(self, x):
        return self.net(x)

In [8]:
STRATEGY_LABELS = {
    0: "Question",
    1: "Reflection of feelings",
    2: "Restatement or Paraphrasing",
    3: "Providing Suggestions",
    4: "Affirmation and Reassurance",
    5: "Information",
    6: "Others"
}
NUM_STRATEGIES = len(STRATEGY_LABELS)

In [9]:
class StrategyHead(nn.Module):
    def __init__(self, hidden_dim, num_labels=NUM_STRATEGIES):
        super().__init__()
        self.net = nn.Sequential(
            nn.LayerNorm(hidden_dim),
            nn.Dropout(0.1),
            nn.Linear(hidden_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, num_labels)
        )

    def forward(self, x):
        return self.net(x)

In [10]:
emotion_ckpt = hf_hub_download(
    repo_id="Aharneish/emotion-head-goemotions",
    filename="emotion_head.pt"
)


emotion_head.pt:   0%|          | 0.00/5.98M [00:00<?, ?B/s]

In [11]:
strategy_ckpt = hf_hub_download(
    repo_id="Aharneish/strategy-head",
    filename="stategy_head.pt"
)


stategy_head.pt:   0%|          | 0.00/5.98M [00:00<?, ?B/s]

In [12]:
hidden_dim = dpo_model.config.hidden_size

In [13]:
emotion_head = EmotionHead(
    hidden_dim=hidden_dim,
    num_labels=28
).to(dpo_model.device)

strategy_head = StrategyHead(
    hidden_dim=hidden_dim,
    num_labels=28
).to(dpo_model.device)


In [14]:
emotion_head.load_state_dict(
    torch.load(emotion_ckpt, map_location=dpo_model.device)
)

strategy_head.load_state_dict(
    torch.load(strategy_ckpt, map_location=dpo_model.device)
)


<All keys matched successfully>

In [15]:
emotion_head.eval()
strategy_head.eval()

StrategyHead(
  (net): Sequential(
    (0): LayerNorm((2880,), eps=1e-05, elementwise_affine=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): Linear(in_features=2880, out_features=512, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.1, inplace=False)
    (5): Linear(in_features=512, out_features=28, bias=True)
  )
)

In [16]:
import torch

def get_reference_scores(example):
    """
    Extract normalized human preference scores from EQ-Bench example.

    Returns:
        dict with:
          - texts: list[str]
          - scores: torch.FloatTensor (normalized)
    """
    # Common field names used across EQ-Bench versions
    responses = (
        example.get("responses")
        or example.get("answers")
        or example.get("candidates")
    )

    if responses is None:
        return None

    texts = []
    raw_scores = []

    for r in responses:
        texts.append(r.get("text") or r.get("response"))

        # Try multiple possible score keys
        score = (
            r.get("score")
            or r.get("human_score")
            or r.get("preference")
            or r.get("rank")
        )

        if score is None:
            continue

        raw_scores.append(float(score))

    if len(raw_scores) == 0:
        return None

    scores = torch.tensor(raw_scores, dtype=torch.float32)

    # Normalize safely to [0, 1]
    min_s, max_s = scores.min(), scores.max()
    if max_s > min_s:
        scores = (scores - min_s) / (max_s - min_s)
    else:
        scores = torch.zeros_like(scores)

    return {
        "texts": texts,
        "scores": scores,
    }


In [17]:
import torch

@torch.no_grad()
def generate_eq_response(
    model,
    tokenizer,
    prompt,
    max_new_tokens=128,
    temperature=0.7,
    top_p=0.9,
):
    model.eval()

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=512,
    ).to(model.device)

    output_ids = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

    # Remove prompt tokens
    generated = output_ids[0][inputs["input_ids"].shape[-1]:]

    return tokenizer.decode(
        generated,
        skip_special_tokens=True
    ).strip()


In [18]:
import torch
import torch.nn.functional as F

@torch.no_grad()
def parse_emotion_scores(
    text,
    model,
    tokenizer,
    emotion_head,
):
    """
    Scores a generated response using the trained emotion head.

    Returns:
        probs: torch.Tensor of shape (num_emotions,)
    """
    model.eval()
    emotion_head.eval()

    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=512,
    ).to(model.device)

    # ----- get pooled features -----
    pooled = get_pooled_features(
        model,
        inputs["input_ids"],
        inputs["attention_mask"],
    )

    # pooled: (1, hidden_dim)
    logits = emotion_head(pooled)

    probs = F.softmax(logits, dim=-1).squeeze(0)

    return probs


In [19]:
def get_pooled_features(model, input_ids, attention_mask):
    """
    Canonical pooled feature extractor for HF / PEFT / Unsloth models
    """
    with torch.no_grad():
        # 🔹 HuggingFace-guaranteed API
        embedding_layer = model.get_input_embeddings()

        embeddings = embedding_layer(input_ids)  # (B, T, H)

        # masked mean pooling
        mask = attention_mask.unsqueeze(-1).float()
        summed = (embeddings * mask).sum(dim=1)
        denom = mask.sum(dim=1).clamp(min=1e-9)

        pooled = summed / denom  # (B, H)

    return pooled


In [20]:
import torch

# Indices from GoEmotions-style labels
# (adjust if your label order differs)
POSITIVE_EMOTIONS = [
    0,   # admiration
    4,   # approval
    5,   # caring
    8,   # desire
    17,  # joy
    18,  # love
    20,  # optimism
    23,  # relief
    27,  # neutral (optional, small weight)
]

def eqbench_score(pred_probs, ref=None):
    """
    Compute EQ-Bench score from predicted emotion probabilities.

    Args:
        pred_probs: torch.Tensor (num_emotions,)
        ref: dict or None (from get_reference_scores)

    Returns:
        float EQ score
    """
    # ---- 1. base emotional intelligence score ----
    pos_mass = pred_probs[POSITIVE_EMOTIONS].sum().item()

    # ---- 2. optional human-alignment term ----
    if ref is not None and "scores" in ref:
        # normalize reference scores
        ref_scores = ref["scores"].to(pred_probs.device)

        # simple cosine similarity for alignment
        if ref_scores.numel() == pred_probs.numel():
            sim = torch.nn.functional.cosine_similarity(
                pred_probs.unsqueeze(0),
                ref_scores.unsqueeze(0),
                dim=-1
            ).item()
        else:
            sim = 0.0
    else:
        sim = 0.0

    # ---- 3. final EQ score ----
    # weight empathy higher than alignment
    eq = pos_mass + 0.3 * sim

    return eq


In [21]:
def run_eqbench_with_heads(
    model,
    tokenizer,
    dataset,
    use_emotion=False,
    use_strategy=False,
    max_samples=None
):
    scores = []

    for i, ex in enumerate(dataset):
        if max_samples and i >= max_samples:
            break

        prompt = ex["prompt"]
        ref = get_reference_scores(ex)

        # ----- generate model output -----
        output = generate_eq_response(model, tokenizer, prompt)
        pred = parse_emotion_scores(
            output,
            model,
            tokenizer,
            emotion_head,
        )

        # ----- base EQ score -----
        eq = eqbench_score(pred, ref)

        # ----- optional head-based reweighting -----
        if use_emotion or use_strategy:
            inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

            with torch.no_grad():
                features = get_pooled_features(
                    model,
                    inputs["input_ids"],
                    inputs["attention_mask"]
                )

            bonus = 0.0

            if use_emotion:
                emo_conf = emotion_head(features).softmax(dim=-1).max().item()
                bonus += 0.1 * emo_conf   # small, controlled bonus

            if use_strategy:
                strat_conf = strategy_head(features).softmax(dim=-1).max().item()
                bonus += 0.1 * strat_conf

            eq = min(eq + bonus, 1.0)

        scores.append(eq)

    return sum(scores) / len(scores)


In [22]:
MAX_SAMPLES = 30   # later set to None for full run

eq_base = run_eqbench_with_heads(
    dpo_model,
    tokenizer,
    eqbench,
    use_emotion=False,
    use_strategy=False,
    max_samples=MAX_SAMPLES
)
print("Eq-base complete\n")
eq_emotion = run_eqbench_with_heads(
    dpo_model,
    tokenizer,
    eqbench,
    use_emotion=True,
    use_strategy=False,
    max_samples=MAX_SAMPLES
)
print("Eq_emotion complete\n")
eq_strategy = run_eqbench_with_heads(
    dpo_model,
    tokenizer,
    eqbench,
    use_emotion=False,
    use_strategy=True,
    max_samples=MAX_SAMPLES
)
print("Eq_strategy complete\n")
eq_full = run_eqbench_with_heads(
    dpo_model,
    tokenizer,
    eqbench,
    use_emotion=True,
    use_strategy=True,
    max_samples=MAX_SAMPLES
)
print("Eq_full compelte\n")

Eq-base complete

Eq_emotion complete

Eq_strategy complete

Eq_full compelte



In [24]:
import pandas as pd

df = pd.DataFrame({
    "Configuration": [
        "No Heads (Baseline)",
        "Emotion Head Only",
        "Strategy Head Only",
        "Emotion + Strategy (Full)"
    ],
    "EQ-Bench Score": [
        eq_base,
        eq_emotion,
        eq_strategy,
        eq_full
    ]
})

df


,Configuration,EQ-Bench Score
0,No Heads (Baseline),0.540432
1,Emotion Head Only,0.584760
2,Strategy Head Only,0.551153
3,Emotion + Strategy (Full),0.606513


In [26]:
df.to_excel("results.xlsx")

In [27]:
MAX_SAMPLES = None  # full dataset

# rerun the four configs


In [ ]:
MAX_SAMPLES = 30   # later set to None for full run

eq_base = run_eqbench_with_heads(
    dpo_model,
    tokenizer,
    eqbench,
    use_emotion=False,
    use_strategy=False,
    max_samples=MAX_SAMPLES
)

eq_emotion = run_eqbench_with_heads(
    dpo_model,
    tokenizer,
    eqbench,
    use_emotion=True,
    use_strategy=False,
    max_samples=MAX_SAMPLES
)

eq_strategy = run_eqbench_with_heads(
    dpo_model,
    tokenizer,
    eqbench,
    use_emotion=False,
    use_strategy=True,
    max_samples=MAX_SAMPLES
)

eq_full = run_eqbench_with_heads(
    dpo_model,
    tokenizer,
    eqbench,
    use_emotion=True,
    use_strategy=True,
    max_samples=MAX_SAMPLES
)


In [ ]:
df2 = pd.DataFrame({
    "Configuration": [
        "No Heads (Baseline)",
        "Emotion Head Only",
        "Strategy Head Only",
        "Emotion + Strategy (Full)"
    ],
    "EQ-Bench Score": [
        eq_base,
        eq_emotion,
        eq_strategy,
        eq_full
    ]
})
df2

In [ ]:
df2.to_excel("results2.xlxs")